# Réseau neuronal convolutif

## Imports

In [3]:
import numpy as np 
import shutil # pour les dossiers
import pandas as pd
import json
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import time
import os
import copy

## Chargement des images

In [4]:
def metrique(path):
    files = os.listdir(path);
    contenu = [];
    for file in files:
        if file.endswith('.json'):
            openFile = open(path+file, "r");
            contenu.append(json.loads(openFile.read())["Image"]);
            openFile.close();
    return contenu;

In [5]:
chemin = "./train/";
contenu = [];
if not os.path.isdir(chemin):
    print('Rajouter le dossier train dans le dossier courant :) ! ');
else :
    contenu = metrique(chemin);
df = pd.DataFrame(contenu)
# on laisse Species car c'est égale au nombre de classe
df.drop(["Genus","ClassId","Family","Vote","Location","Latitude","Longitude","Date","Author","Content","MediaId", "LearnTag", "ImageId2014","ObservationId2014","YearInCLEF","ObservationId"], axis='columns', inplace=True)
df = df.drop_duplicates()
classe  = df.to_numpy();
print(len(classe))


50


In [6]:
X = [];
Y = [];
files = os.listdir("./train")
for file in files:
    if file.endswith('.jpg'):
        X.append(file);
        nameFile = file.split('.')[0];
        for file_class in files:
            nameFileAutre = file_class.split('.')[0];
            if nameFile==nameFileAutre and file_class.endswith('.json'):
                fichierSrc = open("./train/"+file_class, "r");
                contenu = fichierSrc.read();
                monJson = json.loads(contenu)
                Y.append(monJson["Image"]["Species"])
                break;


In [7]:
# on a bien une classe par image !
if(len(X) == len(Y)):
    print("OK")
else:
    print("KO")

OK


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42, stratify=Y)
y_train[0]

'Salix caprea L.'

Mise en place de l'architecture dossier pour avoir les image_datasets

In [10]:
# Création des répertoires ! 
directory = "CNN"
for classe in y_train : 
    if(not os.path.isdir('./CNN/train/'+classe)):
        os.mkdir('./CNN/train/'+classe)
        
for classe in y_test : 
    if(not os.path.isdir('./CNN/val/'+classe)):
        os.mkdir('./CNN/val/'+classe)
        
# on met les images dans les répertoires precédents

# on supprime le contenu

directorys=os.listdir('./CNN/train/')
for i in range(0,len(directory)):
    files=os.listdir('./CNN/train/'+directorys[i])
    for j in range(0,len(files)):
        os.remove('./CNN/train/'+directorys[i]+'/'+files[j])

directorys=os.listdir('./CNN/val/')
for i in range(0,len(directory)):
    files=os.listdir('./CNN/val/'+directorys[i])
    for j in range(0,len(files)):
        os.remove('./CNN/val/'+directorys[i]+'/'+files[j])

# on met le contenu
shutil.copyfile('./train/'+X_train[0],'./CNN/train/'+y_train[0]+'/'+X_train[0])

print('./CNN/train/'+y_train[0]+X_train[0])
print(X_train[0])
for i in range(len(X_train)):
    shutil.copyfile('./train/'+X_train[i],'./CNN/train/'+y_train[i]+'/'+X_train[i])
    
for i in range(len(X_test)):
    shutil.copyfile('./train/'+X_test[i],'./CNN/val/'+y_test[i]+'/'+X_test[i])
    

./CNN/train/Salix caprea L.30108.jpg
30108.jpg


In [17]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}
data_dir = 'CNN'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

class_names = image_datasets['train'].classes
#print(pd.DataFrame(class_names))



In [23]:
model = models.densenet121(pretrained=True)

In [14]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [15]:
model = models.densenet121(pretrained=True)


In [16]:
model = train_model(model, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=25)

NameError: name 'criterion' is not defined